# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [155]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
import re 
import pandas as pd 
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

from nltk.corpus import stopwords
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to /Users/Sigve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Sigve/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Sigve/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('coded_responses', engine)  
X = df["message"]
Y = df.drop(columns=["message","id","genre","original"])

### 2. Write a tokenization function to process your text data

In [160]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [161]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [163]:
Y_pred=pipeline.predict(X_test)

In [164]:
y_pred=pd.DataFrame(data=Y_pred,columns=list(y_test.columns))

In [165]:
for column in y_test:
    print("Column: {}:\n\n".format(column),classification_report(y_test[column].values, y_pred[column].values))

Column: related:

               precision    recall  f1-score   support

           0       0.47      0.03      0.05      1491
           1       0.77      0.99      0.87      5022
           2       0.00      0.00      0.00        39

    accuracy                           0.77      6552
   macro avg       0.41      0.34      0.31      6552
weighted avg       0.70      0.77      0.68      6552

Column: request:

               precision    recall  f1-score   support

           0       0.83      0.99      0.90      5417
           1       0.47      0.02      0.05      1135

    accuracy                           0.83      6552
   macro avg       0.65      0.51      0.47      6552
weighted avg       0.77      0.83      0.76      6552

Column: offer:

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6552
   macro avg       

/miniconda2/envs/dataSci/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Column: transport:

               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6259
           1       0.11      0.01      0.01       293

    accuracy                           0.95      6552
   macro avg       0.53      0.50      0.49      6552
weighted avg       0.92      0.95      0.93      6552

Column: buildings:

               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6205
           1       0.00      0.00      0.00       347

    accuracy                           0.94      6552
   macro avg       0.47      0.50      0.49      6552
weighted avg       0.90      0.94      0.92      6552

Column: electricity:

               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6422
           1       0.00      0.00      0.00       130

    accuracy                           0.98      6552
   macro avg       0.49      0.50      0.49      6552
weighted 

### 6. Improve your model
Use grid search to find better parameters. 

In [166]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1a377f52f0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                                          max_depth=None,
                                                          max_features='auto',
           

In [180]:
parameters = {
        #'vect__ngram_range': ((1, 1), (1, 2)),
        #'vect__max_df': (0.5, 0.75, 1.0),
        #'vect__max_features': (None, 5000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100, 200],
        #'clf__estimator__min_samples_split': [2, 3, 4],
    }

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [179]:
cv.fit(X_train,y_train)


ValueError: multiclass-multioutput is not supported

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [149]:
# save the model to disk
filename = 'test_model.pkl'
joblib.dump(pipeline, filename)

['test_model.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.